In [5]:
from mlflow.tracking import MlflowClient
MLFLOW_SET_TRACKING_URI = "http://127.0.0.1:5000"
client = MlflowClient(tracking_uri=MLFLOW_SET_TRACKING_URI)

In [6]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/2', creation_time=1747069836106, experiment_id='2', last_update_time=1747069836106, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1', creation_time=1746683774913, experiment_id='1', last_update_time=1746683774913, lifecycle_stage='active', name='ny-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/0', creation_time=1746683599358, experiment_id='0', last_update_time=1746683599358, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
client.create_experiment(name="my-cool-experiment")

RestException: RESOURCE_ALREADY_EXISTS: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', '', 'active', 1747131195206, 1747131195206)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [8]:
#List runs on the basis of top 10 lowes rmse with experiment id 1 that is ny_taxi
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
     max_results=5,
    order_by=["metrics.rmse ASC"] #
)

In [ ]:
runs

[<Run: data=<RunData: metrics={'rmse': 5.809681017912009}, params={'learning_rate': '0.09064205157889488',
  'max_depth': '22',
  'min_child_weight': '1.0629059550227644',
  'objective': 'reg:linear',
  'reg_alpha': '0.13507272239403997',
  'reg_lambda': '0.01337017283417198',
  'seed': '42'}, tags={'developer': 'Shakti',
  'mlflow.runName': 'blushing-gnat-647',
  'mlflow.source.name': '/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'codespace',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/bad7bfb0e73340a6bd111039cdfd4375/artifacts', end_time=1746811838007, experiment_id='1', lifecycle_stage='active', run_id='bad7bfb0e73340a6bd111039cdfd4375', run_name='blushing-gnat-647', run_uuid='bad7bfb0e73340a6bd111039cdfd4375', start_time=1746811700264, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset

In [10]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: bad7bfb0e73340a6bd111039cdfd4375, rmse: 5.8097
run id: 022ccf19e9f244548dfe3e4571551726, rmse: 5.8119
run id: 3d51f711bda342be9ef0b6e36113fddc, rmse: 5.8141
run id: 811b89f631de4672a48ceb8ea0526e40, rmse: 5.8236
run id: d3a43abd35a04b18b51f096b004e348b, rmse: 5.8237


In [11]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_SET_TRACKING_URI)

In [34]:
#Found the best runid and registered the model ran on that runid
run_id = "8eea560a668f43cfa378f46d2b4a979d"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/05/13 10:45:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 7
Created version '7' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747133126248, current_stage='None', description='', last_updated_timestamp=1747133126248, name='nyc-taxi-regressor', run_id='8eea560a668f43cfa378f46d2b4a979d', run_link='', source='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/8eea560a668f43cfa378f46d2b4a979d/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='7'>

In [35]:
model_name = "nyc-taxi-regressor"


# Fetch all versions of the model
versions = client.search_model_versions(f"name='{model_name}'")

for v in versions:
    print(f"Version: {v.version}, Status: {v.status}, Stage: {v.current_stage}")


Version: 7, Status: READY, Stage: None
Version: 6, Status: READY, Stage: None
Version: 1, Status: READY, Stage: Production
Version: 2, Status: READY, Stage: None
Version: 5, Status: READY, Stage: None
Version: 4, Status: READY, Stage: None
Version: 3, Status: READY, Stage: None


In [36]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_name = "nyc-taxi-regressor"
model_version = "2"

# Set a tag to indicate stage (e.g., "promoted_to" or "status")
client.set_model_version_tag(
    name=model_name,
    version=model_version,
    key="promoted_to",
    value="staging"
)

# Optionally, update description
client.update_model_version(
    name=model_name,
    version=model_version,
    description="Promoted to staging after evaluation"
)


<ModelVersion: aliases=[], creation_timestamp=1747125264621, current_stage='None', description='Promoted to staging after evaluation', last_updated_timestamp=1747133131246, name='nyc-taxi-regressor', run_id='bad7bfb0e73340a6bd111039cdfd4375', run_link='', source='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/bad7bfb0e73340a6bd111039cdfd4375/artifacts/model', status='READY', status_message=None, tags={'promoted_to': 'staging'}, user_id='', version='2'>

In [37]:
client.transition_model_version_stage(
    name=model_name,
    version="1",  # Replace with the actual version number you want
    stage="Production",
    archive_existing_versions=True
)


/tmp/ipykernel_2651/1523603900.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747116970286, current_stage='Production', description='', last_updated_timestamp=1747133134160, name='nyc-taxi-regressor', run_id='bad7bfb0e73340a6bd111039cdfd4375', run_link='', source='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/bad7bfb0e73340a6bd111039cdfd4375/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [39]:
#Testing if prod model is woking fine with latest data if yes it will be there else it will be replaced

from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np 

def read_dataframe(filename):
    df = pd.read_parquet(filename,engine='pyarrow')

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(model_version, X_test, y_test, model_name='nyc-taxi-regressor'):
    # Load the model using the version number instead of the stage
    model_uri = f"models:/{model_name}/{model_version}"
    model = mlflow.pyfunc.load_model(model_uri)
    
    # Predict using the model
    y_pred = model.predict(X_test)
    
    # Calculate RMSE (Root Mean Squared Error)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return {"rmse": rmse}

In [16]:
df= read_dataframe('./data/green_tripdata_2021-02.parquet')



In [ ]:
#/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/8eea560a668f43cfa378f46d2b4a979d
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoom-camp/03-training/experiment_tracking/preprocessor'

In [ ]:
#Doenload the model preprocess from mlflow to local and a folder is created 
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv= pickle.load(f_in)

In [24]:
#Now test the model downlaoded from mldlow with lateset data

X_test = preprocess(df, dv)

In [25]:
target = "duration"
y_test = df[target].values

In [ ]:
#Test model in prod firts by giving tag,model anme anbd version
model_version = 4
%time test_model(model_version, X_test, y_test, model_name='nyc-taxi-regressor')

OSError: No such file or directory: '/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/8eea560a668f43cfa378f46d2b4a979d/artifacts/model/.'

In [ ]:
#Test model in staging  firts by giving tag,model anme anbd version
model_version = 4
%time test_model(model_version, X_test, y_test, model_name='nyc-taxi-regressor')

In [ ]:
#So we can compare both rmse and think whether we nee dto move the stafging model to porod or not